# Extracción y descripción de los datos

Empiezo importando las librerías que voy a utilizar.

In [2]:
%matplotlib inline

import numpy as np
import pandas as pd

import librosa.display
import librosa

from glob import glob

import ffmpeg

import os

from pydub import AudioSegment
from pydub.utils import make_chunks

import re

Continuo creando 3 funciones con las que voy a hacer los siguiente:
 - La primera divide un archivo audio de larga duración en pequeños audios de 10 segundos de duración, y los guarda en la carpeta que le he especificado.
 - La segunda accede a cada uno de esos audios de 10 segundos y saca las features características de dicho archivo, y en base a su nombre le asigna una categoría y otra.
 - La tercera simplemente me enseña las principales características y métricas de la base de datos.

Finalmente, guardo el el dataset en mi carpeta como un CSV.

Dado que la base de datos ya la tengo creada, este JN lo voy a aplicar sobre audios que luego voy a utilizar para predecir con mi modelo y ver si es lo suficientemente bueno.

In [22]:
def decompose_files(data_dir, audio_files):
    for j in range(len(audio_files)):
        myaudio = AudioSegment.from_file(data_dir + '{}'.format(os.listdir(data_dir)[j])) 
        chunk_length_ms = 2000
        chunks = make_chunks(myaudio, chunk_length_ms)

        for i, chunk in enumerate(chunks):
            chunk_name = "{}{}.wav".format(os.listdir(data_dir)[j], i)
            print("exporting", chunk_name)
            chunk.export('./sounds2/{}'.format(chunk_name), format="mp3")
            
decompose_files('./sounds/', glob('./sounds' + '/*'))

exporting alarm.mp30.wav
exporting alarm.mp31.wav
exporting alarm.mp32.wav
exporting alarm.mp33.wav
exporting alarm.mp34.wav
exporting alarm.mp35.wav
exporting alarm.mp36.wav
exporting alarm.mp37.wav
exporting alarm.mp38.wav
exporting alarm.mp39.wav
exporting alarm.mp310.wav
exporting alarm.mp311.wav
exporting alarm.mp312.wav
exporting alarm.mp313.wav
exporting alarm.mp314.wav
exporting alarm.mp315.wav
exporting alarm.mp316.wav
exporting alarm.mp317.wav
exporting alarm.mp318.wav
exporting alarm.mp319.wav
exporting alarm.mp320.wav
exporting alarm.mp321.wav
exporting alarm.mp322.wav
exporting alarm.mp323.wav
exporting alarm.mp324.wav
exporting alarm.mp325.wav
exporting alarm.mp326.wav
exporting alarm.mp327.wav
exporting alarm.mp328.wav
exporting alarm.mp329.wav
exporting alarm.mp330.wav
exporting alarm.mp331.wav
exporting alarm.mp332.wav
exporting alarm.mp333.wav
exporting alarm.mp334.wav
exporting alarm.mp335.wav
exporting alarm.mp336.wav
exporting alarm.mp337.wav
exporting alarm.mp338.

exporting alarm.mp3308.wav
exporting alarm.mp3309.wav
exporting alarm.mp3310.wav
exporting alarm.mp3311.wav
exporting alarm.mp3312.wav
exporting alarm.mp3313.wav
exporting alarm.mp3314.wav
exporting alarm.mp3315.wav
exporting alarm.mp3316.wav
exporting alarm.mp3317.wav
exporting alarm.mp3318.wav
exporting alarm.mp3319.wav
exporting alarm.mp3320.wav
exporting alarm.mp3321.wav
exporting alarm.mp3322.wav
exporting alarm.mp3323.wav
exporting alarm.mp3324.wav
exporting alarm.mp3325.wav
exporting alarm.mp3326.wav
exporting alarm.mp3327.wav
exporting alarm.mp3328.wav
exporting alarm.mp3329.wav
exporting alarm.mp3330.wav
exporting alarm.mp3331.wav
exporting alarm.mp3332.wav
exporting alarm.mp3333.wav
exporting alarm.mp3334.wav
exporting alarm.mp3335.wav
exporting alarm.mp3336.wav
exporting alarm.mp3337.wav
exporting alarm.mp3338.wav
exporting alarm.mp3339.wav
exporting alarm.mp3340.wav
exporting alarm.mp3341.wav
exporting alarm.mp3342.wav
exporting alarm.mp3343.wav
exporting alarm.mp3344.wav
e

In [43]:
def get_features(data_dir, audio_files):
    momentos = {'ducha': 'Ducha', 'cena': 'Cena', 'washing': 'Lavadora', 'vacuum': 'Aspiradora', 
                'shaver': 'Afeitadora', 'hairdryer': 'Secador_pelo', 'airconditioner': 'Aire_acondicionado', 
                'cellphone': 'Telefono', 'comp': 'Tecleo', 'silence': 'Silencio', 'dryer': 'Secadora', 
                'blender': 'Licuadora', 'doorbell': 'Timbre', 'alarm': 'Alarma', 'faucet': 'Grifo', 
                'microwave': 'Microondas'}
    features = []
    for i in range(len(audio_files)):
        y, sr = librosa.load(audio_files[i], sr=22050, mono=True)
        name = os.listdir(data_dir)[i]
        pattern = "[._][\w]+"
        name = re.sub(pattern, '', name)
        if name in momentos.keys():
            momento = momentos.get(name)
        else:
            momento = 'Otro'
        mfcc = np.ndarray.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20))
        scem = np.ndarray.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        scom = np.ndarray.mean(librosa.feature.spectral_contrast(S=np.abs(librosa.stft(y)), sr=sr, n_bands=4))
        srom = np.ndarray.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
        sbwm = np.ndarray.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
        tempo = librosa.beat.tempo(onset_envelope=librosa.onset.onset_strength(y=y, sr=sr, hop_length=512), sr=sr, hop_length=512)[0]
        rmse = np.ndarray.mean(librosa.feature.rms(y=y))
        D = np.abs(np.fft.fft(y, n=10)[:8000*2 // 2])
        features.append([mfcc, scem, scom, srom, sbwm, tempo, rmse, D[0], D[1], D[2], D[3], D[4], D[5], 
                         D[6], D[7], D[8], D[9], momento])
    return pd.DataFrame(features, columns=['mfcc', 'scem','scom', 'srom','sbwm', 'tempo', 'rmse', 
                                           'Fourier1', 'Fourier2', 'Fourier3', 'Fourier4', 'Fourier5', 
                                           'Fourier6', 'Fourier7', 'Fourier8', 'Fourier9', 'Fourier10', 'momento'])

data = get_features('./sounds2/', glob('./sounds2' + '/*.wav'))

In [44]:
data

,mfcc,scem,scom,srom,sbwm,tempo,rmse,Fourier1,Fourier2,Fourier3,Fourier4,Fourier5,Fourier6,Fourier7,Fourier8,Fourier9,Fourier10,momento
0,-22.985950,3077.218666,18.291471,5742.806270,2154.682757,36.914062,0.056094,2.158041e-01,3.055540e-03,8.417951e-03,5.212146e-03,1.745967e-03,7.399566e-03,1.745967e-03,5.212146e-03,8.417951e-03,3.055540e-03,Alarma
1,-7.262348,4037.978261,21.817861,7532.537210,2886.126529,117.453835,0.118149,8.445145e-03,6.032405e-03,1.746549e-03,3.455654e-03,4.042406e-03,7.436449e-04,4.042406e-03,3.455654e-03,1.746549e-03,6.032405e-03,Alarma
2,-9.443236,4194.341646,21.251744,7651.588598,2876.325355,117.453835,0.117178,5.056380e-01,6.402531e-01,1.461660e-01,4.862224e-01,1.930025e-01,9.601094e-02,1.930025e-01,4.862224e-01,1.461660e-01,6.402531e-01,Alarma
3,-14.762231,3925.654343,20.236956,7284.039096,2759.188381,117.453835,0.091997,6.944763e-01,6.029257e-01,6.895016e-01,4.664728e-01,2.226160e-01,4.425666e-02,2.226160e-01,4.664728e-01,6.895016e-01,6.029257e-01,Alarma
4,-9.279113,4148.781616,21.907621,7639.336948,2874.818569,117.453835,0.116229,4.295553e-02,6.139120e-03,8.567572e-03,9.281534e-03,5.549128e-03,5.176558e-03,5.549128e-03,9.281534e-03,8.567572e-03,6.139120e-03,Alarma
5,-22.898389,3101.376074,18.559974,5799.609375,2159.909054,117.453835,0.056803,5.595265e-04,2.268114e-05,2.084514e-05,2.104824e-05,1.047023e-05,4.430713e-05,1.047023e-05,2.104824e-05,2.084514e-05,2.268114e-05,Alarma
6,-6.397126,4022.755268,21.408253,7531.175916,2890.345043,117.453835,0.119676,4.586905e-01,1.166674e+00,1.418072e-01,2.226562e-01,2.324754e-01,1.570493e-03,2.324754e-01,2.226562e-01,1.418072e-01,1.166674e+00,Alarma
7,-9.525356,4177.814594,21.340952,7641.069504,2873.086489,123.046875,0.116766,5.065500e-02,5.594293e-03,3.255714e-03,1.055940e-02,5.687337e-03,5.770104e-04,5.687337e-03,1.055940e-02,3.255714e-03,5.594293e-03,Alarma
8,-9.308346,4147.296774,21.996335,7617.308728,2872.771891,117.453835,0.116322,4.410341e-02,5.294883e-03,8.679025e-03,9.608370e-03,3.148405e-03,5.856698e-03,3.148405e-03,9.608370e-03,8.679025e-03,5.294883e-03,Alarma
9,-9.438584,4191.698437,21.308250,7644.410863,2877.935167,117.453835,0.117418,5.090122e-01,6.265712e-01,2.582324e-01,5.794569e-01,2.281116e-01,1.369392e-01,2.281116e-01,5.794569e-01,2.582324e-01,6.265712e-01,Alarma


In [47]:
data_alt = data[data['Fourier5'] != 0]

In [48]:
data_alt[data_alt['Fourier5'] == 0]

,mfcc,scem,scom,srom,sbwm,tempo,rmse,Fourier1,Fourier2,Fourier3,Fourier4,Fourier5,Fourier6,Fourier7,Fourier8,Fourier9,Fourier10,momento


In [46]:
data.describe()

,mfcc,scem,scom,srom,sbwm,tempo,rmse,Fourier1,Fourier2,Fourier3,Fourier4,Fourier5,Fourier6,Fourier7,Fourier8,Fourier9,Fourier10
count,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000
mean,-10.312899,4008.277586,20.954905,7387.158317,2795.017846,116.167294,0.109255,0.344227,0.330436,0.182743,0.183696,0.114038,0.030733,0.114038,0.183696,0.182743,0.330436
std,4.060308,290.019242,0.885593,528.644235,211.138154,14.136493,0.017034,0.435237,0.394935,0.229203,0.216392,0.128409,0.047401,0.128409,0.216392,0.229203,0.394935
min,-23.105433,2947.191723,17.991425,5329.096511,1977.107995,36.914062,0.055766,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-9.656688,4023.197301,20.939618,7503.207503,2853.216105,117.453835,0.115653,0.019292,0.005926,0.003811,0.004697,0.003393,0.001612,0.003393,0.004697,0.003811,0.005926
50%,-9.172000,4104.063456,21.203728,7545.655139,2869.987450,117.453835,0.116717,0.063875,0.014274,0.008655,0.011680,0.007180,0.005224,0.007180,0.011680,0.008655,0.014274
75%,-8.541408,4158.967848,21.487418,7618.855654,2878.461322,117.453835,0.117617,0.568106,0.664518,0.354261,0.374597,0.235738,0.043737,0.235738,0.374597,0.354261,0.664518
max,-5.895563,4229.816411,22.300036,7839.694740,2982.233360,198.768029,0.119760,1.477407,1.220708,0.844557,0.685030,0.411593,0.239982,0.411593,0.685030,0.844557,1.220708


In [21]:
def show_df_info(dataframe):
    display("The type of the dataframe is {}.".format(type(dataframe)))
    print("------------------------------------------------------------------------")
    display("The amount of entries are {}.".format(dataframe.size))
    print("------------------------------------------------------------------------")
    display("Its shape is {}.".format(dataframe.shape))
    print("------------------------------------------------------------------------")
    display("Its features are: {}.".format(list(dataframe)))
    print("------------------------------------------------------------------------")
    print("The data types of columns are: {}".format(dataframe.dtypes))
    print("------------------------------------------------------------------------")
    print("The counter of each class is: {}".format(dataframe.momento.value_counts()))
    print("------------------------------------------------------------------------")
    print('The amount of null values is: {}.'.format(dataframe.isna().sum()))
    display(dataframe.head())
    display(dataframe.describe())
    
show_df_info(data)

"The type of the dataframe is <class 'pandas.core.frame.DataFrame'>."

------------------------------------------------------------------------


'The amount of entries are 224.'

------------------------------------------------------------------------


'Its shape is (28, 8).'

------------------------------------------------------------------------


"Its features are: ['mfcc', 'scem', 'scom', 'srom', 'sbwm', 'tempo', 'rmse', 'momento']."

------------------------------------------------------------------------
The data types of columns are: mfcc       float64
scem       float64
scom       float64
srom       float64
sbwm       float64
tempo      float64
rmse       float64
momento     object
dtype: object
------------------------------------------------------------------------
The counter of each class is: Grifo         14
Aspiradora    14
Name: momento, dtype: int64
------------------------------------------------------------------------
The amount of null values is: mfcc       0
scem       0
scom       0
srom       0
sbwm       0
tempo      0
rmse       0
momento    0
dtype: int64.


,mfcc,scem,scom,srom,sbwm,tempo,rmse,momento
0,-1.209925,3899.153153,18.999687,7674.313687,3091.657072,123.046875,0.153011,Aspiradora
1,-0.073909,4462.048199,19.394828,8128.784180,3059.540310,120.000000,0.225774,Aspiradora
2,0.091820,4143.076863,16.099284,8194.982542,3002.130473,123.046875,0.106121,Grifo
3,-1.693803,4342.919398,19.224401,7870.685508,3063.301995,123.046875,0.144743,Aspiradora
4,-9.215146,4809.931879,19.011070,8420.756428,3036.757352,123.046875,0.216905,Aspiradora


,mfcc,scem,scom,srom,sbwm,tempo,rmse
count,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000
mean,-4.303088,4307.866445,17.994960,8092.243933,3014.115586,123.290762,0.121649
std,3.919068,314.678288,1.655027,404.215899,79.837520,2.842506,0.066696
min,-11.506991,3607.017904,15.780858,6607.970483,2733.730116,117.453835,0.024315
25%,-8.202848,4125.403339,16.238359,8027.922232,3002.659434,123.046875,0.054407
50%,-3.896869,4202.316503,18.727741,8236.625059,3034.369521,123.046875,0.109756
75%,-0.690039,4491.074084,19.035133,8311.685259,3040.364474,123.046875,0.167872
max,1.234239,4822.410675,21.697894,8447.185812,3184.399716,129.199219,0.225774


In [22]:
#data.to_csv('data_prueba.csv', index=False)

In [ ]:
data.to_csv('data_fourier.csv', index=False)